<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/tf_agents_ppo_trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!pip install tf_agents
!pip install yfinance
!pip install tensorflow-probability


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [70]:
from google.colab import drive
drive.mount('/content/drive')

import sys

sys.path.insert(0,'/content/drive/My Drive/ml-trde-notebooks')
data_path = '/content/drive/My Drive/ml-trde-notebooks/data'
model_path = "/content/drive/My Drive/ml-trde-notebooks/models"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tf_agents
from tf_agents.environments import suite_gym, tf_py_environment
from tf_agents.networks import network, utils, categorical_projection_network
from tf_agents.agents.ppo import ppo_agent
from tf_agents.policies import actor_policy
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.agents import tf_agent
from tf_agents.networks import actor_distribution_network, value_network
from tf_agents.utils import common
from tf_agents.environments import parallel_py_environment

import tensorflow_probability as tfp

from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import norm

import multiprocessing
from itertools import cycle

cores = multiprocessing.cpu_count()
print("I has cores: ", cores)
try:
  tf_agents.system.multiprocessing.enable_interactive_mode()
except Exception as inst:
  pass




I has cores:  2


In [72]:
# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/backtest-common.ipynb'

Modified
1679185329
1679185329.0
1679185329.0


In [73]:
logging_level = 2
def error(*args):
    if logging_level >= 0:
        print(*args)

def info(*args):
    if logging_level >=1:
        print(*args)

def verbose(*args):
    if logging_level >=2:
        print(*args)

def debug(*args):
    if logging_level >=3:
        print(*args)

In [74]:
def download_stocks():
  # We don't want to train against crypto
  return get_all_stock_timerseries_for_csv("training_tickers3.csv", 3500, 10)

LENGTH_OF_STOCK_TRAINGING_DATA = 145 #I might need to fix this, but the model is tied to the number of symbols we trained on
def download_crypto():
  return get_all_product_timeseries(-1, 180, LENGTH_OF_STOCK_TRAINGING_DATA)

In [75]:
from_disk = False
use_crypto = False

if from_disk:
  product_data = load_dict(use_crypto)
  first_key = list(product_data.keys())[0]
  length = len(product_data[first_key])
elif use_crypto:
  product_data = download_crypto()
  save_dict(product_data, True)
else:
  product_data = download_stocks()
  save_dict(product_data)

min_len = min(len(df) for df in product_data.values())
print("symbols:", len(product_data.keys()))
print("min length:", min_len)
for name, df in product_data.items():
    product_data[name] = df.head(min_len)

loading ticker:  XTSLA  count: 0 of: 164
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- XTSLA: No timezone found, symbol may be delisted
loading ticker:  XEL  count: 1 of: 164
[*********************100%***********************]  1 of 1 completed
loading ticker:  WU  count: 2 of: 164
[*********************100%***********************]  1 of 1 completed
loading ticker:  WTW  count: 3 of: 164
[*********************100%***********************]  1 of 1 completed
loading ticker:  WST  count: 4 of: 164
[*********************100%***********************]  1 of 1 completed
loading ticker:  WMT  count: 5 of: 164
[*********************100%***********************]  1 of 1 completed
loading ticker:  WMB  count: 6 of: 164
[*********************100%***********************]  1 of 1 completed
loading ticker:  WM  count: 7 of: 164
[*********************100%***********************]  1 of 1 completed
loading ticker:  WEC  count: 8 of: 164
[*********************100%

In [86]:
import numpy as np
from gym import spaces
from tf_agents.environments import py_environment
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts

class TraderEnv(py_environment.PyEnvironment):
    
    def __init__(self, product, df):
        super(TraderEnv, self).__init__()

        # Define observation spec
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(4,), dtype=np.float32, minimum=0, maximum=np.inf, name='observation'
        )

        # Define action spec
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=1, name='action'
        )

        # Initialize environment state
        df = self.expand(df.copy())
      
        self.timeseries = self.scale(df[["Date", "Close", "weighted-volume", "trend", "prob_above_trend"]])
        self.product = product
        self.final = len(df)
        self._reset_vars()
    
    def expand(self, df):
      # Perform seasonal decomposition
      result = seasonal_decompose(df['Close'], model='additive', period=90, extrapolate_trend='freq')

      # Add trend back to original time series
      df["trend"] = result.trend

      # Compute the residuals by subtracting the trend from the original time series
      residuals = result.resid

      # Fit a Gaussian distribution to the residuals
      mu, std = norm.fit(residuals)

      # Compute the probability of a value being above or below the trend line
      # for each point in the time series
      z_scores = residuals / std
      df["prob_above_trend"] = 1 - norm.cdf(z_scores)
      df["weighted-volume"] = df["Close"] * df["Volume"]
      return df

    def scale(self, timeseries):
      df = timeseries.reset_index()  # Reset the index of the DataFrame
      dates = df['Date']
      data_to_scale = df.drop('Date', axis=1)
      self.scaler = MinMaxScaler()
      scaled_data = self.scaler.fit_transform(data_to_scale)
      scaled_df = pd.concat([dates, pd.DataFrame(scaled_data, columns=data_to_scale.columns)], axis=1)
      return scaled_df.set_index('Date')  # Set the index back to 'Date'

    def make_ledger_row(self):
        ledger = pd.DataFrame()
        ledger["Date"] = []
        ledger["Side"] = []
        ledger["Action"]  = []
        ledger["Price"] = []
        ledger["Fee"] = []
        return ledger

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset_vars(self):
        self._state = None
        self._episode_ended = False        
        self.ledger = self.make_ledger_row()
        self.slippage = .01
        self.fee = .0025
        self.current_index = 0
        self.cash = 5000
        self.initial_cash = self.cash
        self.position_shares = 0
        self.position_value = 0
        self._state = self._get_initial_state()

    def _reset(self):
        # Reset the environment and return the initial time step
        self._reset_vars()
        return ts.restart(self._state)

    def _step(self, action):
        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start a new episode.
            return self.reset()

        # Apply the action and update the environment state
        self._apply_action(action)
        self._state = self._get_next_state()

        if self._is_episode_ended():
            self._episode_ended = True
            reward = self._get_final_reward()
            return ts.termination(self._state, reward)
        else:
            reward = self._get_reward()
            return ts.transition(self._state, reward=reward, discount=1.0)

    def _get_initial_state(self):
        # Return the initial state of the environment
        self.current_index = 0
        return self.env_block()

    def _apply_action(self, action):
        # Advance the environment by one time step and return the observation, reward, and done flag
        self.current_index += 1
        info("step:", "index:", self.current_index, " of: ", self.final-1, " cash: ", self.cash, " value: ", self.position_value)

        if (self.current_index >= self.final - 1 or  self._get_reward() <= -1):
          info("********MARKING DONE", "index:", self.current_index, " of: ", self.final-1, " cash: ", self.cash, " value: ", self.position_value)
          if (self.position_shares != 0):
            info("done so closing position")
            self.close_position()
          self._episode_ended = True
        else:
          self._episode_ended = False

    
        if (action == 1 and self.position_shares != 0):
          info("holding: ")
          self.update_position_value()
        elif (action == 0):
          info("closing: ")
          self.close_position()
        elif (action == 1 and self.position_shares == 0):
          info("opening: ")
          self.open_position()
        else: #assume hold
          self.update_position_value()

    def _get_next_state(self):
        # Calculate and return the next state based on the current state and action taken
        return self.env_block()

    def update_position_value(self):
        df = self.timeseries
        row = df.iloc[self.current_index,:]
        self.position_value = row["Close"] * self.position_shares

    def _is_episode_ended(self):
        return self.current_index >= self.final
        

    def _get_reward(self):
        self.update_position_value()
        current_portfolio_value = self.position_value + self.cash
        percentage_change = (current_portfolio_value - self.initial_cash) / self.initial_cash
        clipped_reward = np.clip(percentage_change, -1, np.inf)
        return clipped_reward

    def _get_final_reward(self):
        # Calculate and return the final reward when the episode ends
        return self._get_reward()

    def env_block(self):
      start_index = self.current_index
      end_index = self.current_index
      df = self.timeseries.copy()
      df = self.timeseries.reset_index().drop(['Date'], axis=1)
      block = df.iloc[start_index].to_numpy()
      return block


In [91]:
env_count = 0
num_parallel_envs = 4
learning_rate = 3e-4
num_epochs = 25
discount_factor = 0.99
gradient_clipping = 0.5
entropy_regularization = 1e-2
value_pred_loss_coef = 0.5
use_gae = True
use_td_lambda_return = True
actor_loss_weight = 1.0
value_loss_weight = 0.5
log_interval = 50
eval_interval = 100

def create_env():
    global env_count
    products = list(product_data.keys())
    product = products[env_count]
    env = TraderEnv(product, product_data[product])
    env_count+=1
    return env

def collect_episode(environment, policy):
    episode = []
    time_step = environment.reset()
    while not time_step.is_last():
        action_step = policy.action(time_step)
        next_time_step = environment.step(action_step.action)
        episode.append((time_step, action_step, next_time_step.reward))
        time_step = next_time_step
    return episode

def train_one_episode(agent, environment):
    episode = collect_episode(environment, agent.collect_policy)
    experience = to_trajectories(episode)
    train_loss = agent.train(experience=experience)
    return train_loss, sum([step[2] for step in episode])

def to_trajectories(episode):
    time_steps, actions, rewards = zip(*episode)
    return trajectory.Trajectory(
        observation=tf.nest.map_structure(np.stack, [t.observation for t in time_steps]),
        action=tf.nest.map_structure(np.stack, [a.action for a in actions]),
        policy_info=(),
        reward=tf.stack(rewards),
        discount=tf.constant([1.0] * len(rewards), dtype=tf.float32),
        step_type=tf.nest.map_structure(np.stack, [t.step_type for t in time_steps]),
        next_step_type=tf.nest.map_structure(np.stack, [t.step_type for t in time_steps[1:]] + [time_steps[-1].step_type]),
    )

def lstm_layers(input, lstm_units=64, dropout_rate=0.3):
    
    batch_size = input.shape[0]  # Get the batch size from the input tensor
    features = input.shape[1]   # Get the number of features from the input tensor
    timestep = 1  # Set the time step dimension size

    # Reshape the input tensor to have an additional time step dimension
    x = tf.keras.layers.Reshape((batch_size, features))(input)

    x = tf.keras.layers.LSTM(lstm_units, return_sequences=True)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(lstm_units, return_sequences=True)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(lstm_units)(x)
    
    # Add the final dense layer with sigmoid activation
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    return x


def compute_avg_return(environment, policy, num_episodes=10):
    total_return = 0.0
    for _ in range(num_episodes):
        time_step = environment.reset()
        episode_return = 0.0
        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

class ActorNetworkWithLSTM(network.Network):

    def __init__(self, observation_spec, action_spec, lstm_units=64):
        super(ActorNetworkWithLSTM, self).__init__(
            input_tensor_spec=observation_spec,
            state_spec=(),
            name='ActorNetworkWithLSTM'
        )
        self._lstm_units = lstm_units
        self._input_layer = tf.keras.layers.InputLayer(input_shape=observation_spec.shape)
        self._lstm_layers = lstm_layers
        
        

    def call(self, observations, step_type=None, network_state=()):
        x = self._input_layer(observations)
        logits = self._lstm_layers(x, lstm_units=self._lstm_units)
        discrete_actions = tf.cast(output > 0.5, dtype=tf.int32)
        return tf.squeeze(discrete_actions, axis=-1), network_state
        return logits, network_state


class ValueNetworkWithLSTM(network.Network):

    def __init__(self, observation_spec, lstm_units=64):
        super(ValueNetworkWithLSTM, self).__init__(
            input_tensor_spec=observation_spec,
            state_spec=(),
            name='ValueNetworkWithLSTM'
        )

        self._lstm_units = lstm_units
        self._input_layer = tf.keras.layers.InputLayer(input_shape=observation_spec.shape)
        self._lstm_layers = lstm_layers
        self._dense_layer = tf.keras.layers.Dense(1)

    def call(self, observations, step_type=None, network_state=()):
        x = self._input_layer(observations)
        x = self._lstm_layers(x, lstm_units=self._lstm_units)
        value = self._dense_layer(x)
        return tf.reshape(value, [-1]), network_state


# Create the environment
all_envs = [create_env() for _ in range(len(product_data))]

train_parallel_py_envs = [lambda: env for env in all_envs]
train_parallel_env = parallel_py_environment.ParallelPyEnvironment(train_parallel_py_envs)

train_env = tf_agents.environments.tf_py_environment.TFPyEnvironment(train_parallel_env)

# Create the actor and value networks
actor_net = ActorNetworkWithLSTM(train_env.observation_spec(), train_env.action_spec())
value_net = ValueNetworkWithLSTM(train_env.observation_spec())

# Define optimizer, PPO hyperparameters, and create the PPO agent
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
clip_rho_threshold = 0.2
clip_pg_threshold = 0.2

tf_agent = tf_agents.agents.PPOAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    optimizer,
    actor_net=actor_net,
    value_net=value_net,
    num_epochs=num_epochs,
    discount_factor=discount_factor,
    importance_ratio_clipping=clip_pg_threshold,
    entropy_regularization=entropy_regularization,
    gradient_clipping=gradient_clipping,
    value_pred_loss_coef=value_pred_loss_coef,
    use_gae=True,
)

# Initialize the policy
tf_agent.initialize()

# Main training loop
num_iterations = 1000
collect_episodes_per_iteration = 50

tf_agent.train_step_counter.assign(0)

for _ in range(num_iterations):
    # Collect a few episodes using collect_policy and save to the replay buffer.
    total_loss = 0
    for _ in range(collect_episodes_per_iteration):
        train_loss = train_one_episode(tf_agent, train_env)
        total_loss += train_loss.loss.numpy()
    
    avg_loss = total_loss / collect_episodes_per_iteration

    step = tf_agent.train_step_counter.numpy()

    if step % log_interval == 0:
        print(f'step = {step}: loss = {avg_loss}')



policy = tf_agent.policy
tf.saved_model.save(policy, model_path + "/ppo")

ValueError: ignored

In [ ]:
#predict
# Load the saved policy
load = False
if load:
  loaded_policy = tf.saved_model.load(model_path + "/ppo")
def predict(policy_path, env, loaded_policy, num_steps=100):
    
    # Reset the environment to get the initial observation
    time_step = env.reset()

    for step in range(num_steps):
        # Use the loaded policy to choose an action based on the observation
        action_step = loaded_policy.action(time_step)

        # Execute the chosen action in the environment
        next_time_step = env.step(action_step.action)

        # Extract the observation, reward, and other information
        observation = next_time_step.observation
        reward = next_time_step.reward
        done = next_time_step.is_last()

        print(f"Step {step + 1}:")
        print(f"Observation: {observation}")
        print(f"Action: {action_step.action}")
        print(f"Reward: {reward}")
        print(f"Done: {done}")
        print()

        # Update the time_step for the next iteration
        time_step = next_time_step

        # Break the loop if the episode is finished
        if done:
            break


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
